In [114]:
import pandas as pd
share=pd.read_csv('/home/arjun/Downloads/new_share.csv')
share.dropna(axis=1,inplace=True)
emotions=pd.read_csv('/home/arjun/Downloads/EMOTIONS.csv')
share
emotions
# change the date formats of share and emotions
emotions['Date']=pd.to_datetime(emotions['Date']).dt.strftime('%d/%m/%Y')
emotions['Date']=pd.to_datetime(emotions.Date)
emotions['month']=emotions.Date.dt.month
emotions['year']=emotions.Date.dt.year
share['Date']=pd.to_datetime(share['Date']).dt.strftime('%d/%m/%Y')
share['Date']=pd.to_datetime(share.Date)
share['month']=share.Date.dt.month
share['year']=share.Date.dt.year
# merge emotions and share 
share_emo=emotions.merge(share,on='Date',how='inner')
emotion_names=share_emo.columns
# get emotions 
emotion_names=[i  for i in emotion_names if i.find('-E-')>1]
# create a data frame to store values
emo_df_corr_daily=pd.DataFrame()
#subbset df and run loop
for row_num,emo in enumerate(emotion_names):
    temp_df=share_emo[[emo,'z_norm']]
    corr_val=temp_df.corr().iloc[0,1]
    emo_df_corr_daily.loc[row_num,'metric']=emo
    emo_df_corr_daily.loc[row_num,'cor_value']=corr_val
# monthly correlation done



In [115]:
emo_df_corr_daily

,metric,cor_value
0,anger-E-1,0.046310
1,disgust-E-1,-0.003626
2,fear-E-1,0.019479
3,joy-E-1,-0.096157
4,sadness-E-1,0.071228
5,surprise-E-1,0.065751


In [119]:
    ######################################### MONTHLY CORRELATION #####################

### STEP 1. CREATE EMOTION LEVEL DATA ON MONTHLY LEVEL

emo_melt=pd.melt(emotions,id_vars=['Date','month','year'],var_name='emotion')
emo_melt_gb_month=emo_melt.groupby(['year','month','emotion'])['value'].mean().reset_index()    
emo_monthly_mean=emo_melt_gb_month.pivot_table('value',['year','month'],'emotion').reset_index()
#emo_monthly_mean.drop('emotion',axis=1,inplace=True)

### STEP 2. CREATE SHARE LEVEL DATA ON MONTHLY LEVEL

# share groupby monthly level
share_gb_month=share.groupby(['year','month'])['share'].mean().reset_index()
#z_norm_calc monthly
mean_share_month=share_gb_month.share.mean()
stddev_share_month=share_gb_month.share.std()

share_gb_month['z_norm_monthly']=share_gb_month.share.apply(lambda x: (x-mean_share_month)/stddev_share_month  )


### STEP 3. MERGE share_gb_month AND emo_monthly_mean DATA ON MONTH AND YEAR KEY

share_emo_month_joined=share_gb_month.merge(emo_monthly_mean,on=['year','month'],how='inner')

### STEP 4 . APPLY LOOP ON ALL EMOTIONS AND STORE THE CORR DATA 
emotion_names=share_emo_month_joined.columns
# get emotions 
emotion_names=[i  for i in emotion_names if i.find('-E-')>1]
# create a data frame to store values
emo_df_corr_monthly=pd.DataFrame()
#subbset df and run loop


# monthly correlation done




In [120]:
for row_num,emo in enumerate(emotion_names):
    temp_df=share_emo_month_joined[[emo,'z_norm']]
    corr_val=temp_df.corr().iloc[0,1]
    emo_df_corr_monthly.loc[row_num,'metric']=emo
    emo_df_corr_monthly.loc[row_num,'cor_value']=corr_val